In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize']= (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
od.download('https://www.kaggle.com/jsphyg/weather-dataset-rattle-package')

In [ ]:
os.listdir('weather-dataset-rattle-package')

In [ ]:
raw_df = pd.read_csv('weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
raw_df

In [ ]:
raw_df.info()

In [ ]:
raw_df.dropna(subset=['RainTomorrow'], inplace=True)

In [ ]:
plt.title('No. of rows per year')
sns.countplot(x= pd.to_datetime(raw_df['Date']).dt.year)

In [ ]:
year = pd.to_datetime(raw_df.Date).dt.year

train_df = raw_df[year < 2015]
val_df = raw_df[year == 2015]
test_df = raw_df[year > 2015]

In [ ]:
train_df.shape, val_df.shape, test_df.shape


In [ ]:
input_cols = list(train_df.columns)[1:-1]
target_col = 'RainTomorrow'

In [ ]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

In [ ]:
numeric_col = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_col = train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
train_inputs[numeric_col].isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy='mean').fit(raw_df[numeric_col])

In [ ]:
train_inputs[numeric_col] = imputer.transform(train_inputs[numeric_col])
val_inputs[numeric_col] = imputer.transform(val_inputs[numeric_col])
test_inputs[numeric_col] = imputer.transform(test_inputs[numeric_col])

In [ ]:
val_inputs[numeric_col].isna().sum()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler().fit(raw_df[numeric_col])

In [ ]:
train_inputs[numeric_col] = scaler.transform(train_inputs[numeric_col])
val_inputs[numeric_col] = scaler.transform(val_inputs[numeric_col])
test_inputs[numeric_col] = scaler.transform(test_inputs[numeric_col])

In [ ]:
val_inputs.describe().loc[['min', 'max']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_df[categorical_col].fillna('Unknown').count()

In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(raw_df[categorical_col])

In [ ]:
encoded_col = list(encoder.get_feature_names_out(categorical_col))

In [ ]:
train_inputs[encoded_col] = encoder.transform(train_inputs[categorical_col])
val_inputs[encoded_col] = encoder.transform(val_inputs[categorical_col])
test_inputs[encoded_col] = encoder.transform(test_inputs[categorical_col])

In [ ]:
encoded_col

In [ ]:
test_inputs[encoded_col]

In [ ]:
X_train = train_inputs[numeric_col + encoded_col]
X_val = val_inputs[numeric_col + encoded_col]
X_test = test_inputs[numeric_col + encoded_col]

In [ ]:
X_test

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(random_state = 42)
model.fit(X_train, train_targets)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
train_preds = model.predict(X_train)

In [ ]:
train_preds

In [ ]:
pd.value_counts(train_preds)

In [ ]:
train_probs = model.predict_proba(X_train)

In [ ]:
train_probs

In [ ]:
accuracy_score(train_targets, train_preds)

In [ ]:
model.score(X_val, val_targets)

In [ ]:
val_targets.value_counts()/len(val_targets)

In [ ]:
from sklearn.tree import plot_tree, export_text

In [ ]:
plt.figure(figsize=(80, 20))
plot_tree(model, feature_names  = X_train.columns, max_depth =2, filled=True, fontsize=14)

In [ ]:
model.tree_.max_depth

In [ ]:
tree_text = export_text(model, max_depth = 10, feature_names=list(X_train.columns))
print(tree_text)

In [ ]:
model.feature_importances_

In [ ]:
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
})

In [ ]:
importance_df

In [ ]:
plt.title('Feature Importance')
sns.barplot(importance_df.head(10), x='importance', y='feature')

In [ ]:
?DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(max_depth = 3, random_state = 42)

In [ ]:
model.fit(X_train, train_targets)

In [ ]:
model.score(X_train, train_targets)

In [ ]:
model.score(X_val, val_targets)

In [ ]:
model.classes_

In [ ]:
plt.figure(figsize=(80, 20))
plot_tree(model, feature_names  = X_train.columns, max_depth =3, filled=True, fontsize=14)


In [ ]:
print(export_text(model, feature_names=list(X_train.columns)))

In [ ]:
def max_depth_error(md):
  model = DecisionTreeClassifier(max_depth=md, random_state= 42)
  model.fit(X_train, train_targets)
  train_acc = model.score(X_train, train_targets)
  val_acc = model.score(X_val, val_targets)
  return {'Max Depth': md, 'Training Error': train_acc, 'Validation Error': val_acc}

In [ ]:
error_df = pd.DataFrame([max_depth_error(md) for md in range(1,21)])

In [ ]:
error_df

In [ ]:
plt.figure()
plt.plot(error_df['Max Depth'], error_df['Training Error'])
plt.plot(error_df['Max Depth'], error_df['Validation Error'])
plt.title('Training vs. Validation Error')
plt.xticks(range(0,21))
plt.xlabel('Max. Depth')
plt.ylabel('Prediction Error (1 - Accuracy)')
plt.legend(['Training', 'Validation'])

In [ ]:
model = DecisionTreeClassifier(max_depth= 7, random_state = 42).fit(X_train, train_targets)

In [ ]:
model.score(X_val, val_targets)

In [ ]:
model= DecisionTreeClassifier(max_leaf_nodes = 128, random_state = 42)

In [ ]:
model.fit(X_train, train_targets)

In [ ]:
model.score(X_train,train_targets)

In [ ]:
model.score(X_val, val_targets)

In [ ]:
model_text = export_text(model, feature_names=list(X_train.columns))
print(model_text)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs = -1, random_state = 42)



In [ ]:
model.fit(X_train, train_targets)

In [ ]:
model.score(X_train, train_targets)


In [ ]:
model.score(X_val, val_targets)

In [ ]:
train_probs = model.predict_proba(X_train)


In [ ]:
X_train.shape

In [ ]:
train_probs.shape

In [ ]:
model.estimators_[0]

In [ ]:
plt.figure(figsize=(80,20))
plot_tree(model.estimators_[0], max_depth=2, feature_names=X_train.columns, filled=True, rounded=True, class_names=model.classes_);

In [ ]:
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
importance_df

In [ ]:
sns.barplot(importance_df.head(10), x='importance', y='feature')

In [ ]:
?RandomForestClassifier

In [ ]:
base_model = RandomForestClassifier(random_state = 42, n_jobs=-1).fit(X_train, train_targets)

In [ ]:
base_model.score(X_train, train_targets)


In [ ]:
base_model.score(X_val, val_targets)

In [ ]:
model = RandomForestClassifier(random_state = 42, n_jobs=-1, n_estimators=10).fit(X_train, train_targets)

In [ ]:
model.score(X_train, train_targets), model.score(X_val, val_targets)

In [ ]:
model = RandomForestClassifier(random_state = 42, n_jobs=-1, n_estimators=500).fit(X_train, train_targets)

In [ ]:
model.score(X_train, train_targets), model.score(X_val, val_targets)

In [ ]:
def test_params(**params):
  model = RandomForestClassifier(random_state = 42, n_jobs=-1, **params).fit(X_train, train_targets)
  return model.score(X_train, train_targets), model.score(X_val, val_targets)

In [ ]:
test_params(max_features = 'log2')

In [ ]:
test_params(max_features = 3)

In [ ]:
test_params(min_samples_split=3, min_samples_leaf=2)

In [ ]:
test_params(min_impurity_decrease=1e-6)

In [ ]:
test_params(bootstrap = False)

In [ ]:
test_params(max_samples = 0.9)

In [ ]:
model.classes_

In [ ]:
test_params(class_weight ='balanced')

In [ ]:
test_params(class_weight ={'No': 1, 'Yes': 2})

In [ ]:
model = RandomForestClassifier(n_jobs = -1, random_state = 42, n_estimators= 500, max_features = 7, max_depth = 30, class_weight={'No': 1, 'Yes':1.5})

In [ ]:
model.fit(X_train, train_targets)

In [ ]:
model.score(X_train, train_targets), model.score(X_val, val_targets)

In [ ]:
model.score(X_test, test_targets)

In [ ]:
def predcit_input(single_input):
  input_df = pd.DataFrame([single_input])
  input_df[nnumeric_cols] = imputer.transform(input_df[numeric_cols])
  input_df[numeric_cols] = scaler.transform(input_df[numeric_cols])
  input_df[encoded_cols] = encoder.transform(input_df[categorical_cols])
  X_input = input_df[numeric_cols + encoded_cols]
  pred - model.predict(X_input)[0]
  prob = model.predict_prob(X_input)[0][list(model.classes_).index(pred)]

  return {'Prediction': pred, 'Probability': prob}